# Stacking ensembling

Vamos a crear un ensamble de tipo stacking. Para ello se utilizaron las siguientes fuentes:
 - https://mlwave.com/kaggle-ensembling-guide/
 - http://blog.kaggle.com/2016/12/27/a-kagglers-guide-to-model-stacking-in-practice/
 - https://github.com/emanuele/kaggle_pbr/blob/master/blend.py

Para ello se desarrollaron las siguientes secciones:
 - [Predictores base](#Predictores-base)
     - [Xgboost](#Xgboost)
     - [Random Forest](#Random-Forest)
     - [AdaBoost](#AdaBoost)
 - [Metafeatures](#Metafeatures)
 - [Predictor Stacking](#Predictor-Stacking)

In [1]:
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from bayes_opt import BayesianOptimization
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
from time import time
import xgboost as xgb
import pandas as pd
import os

In [2]:
if '__file__' in locals():
    current_folder = os.path.dirname(os.path.abspath(__file__))
else:
    current_folder = os.getcwd()

set_de_entrenamiento_testing_y_prediccion = '"{}"'.format(os.path.join(
    current_folder,
    '..',
    'Set de entrenamiento, testing y predicción.ipynb'
))
merge_features = '"{}"'.format(os.path.join(current_folder, '..', 'Features', 'Merge features.ipynb'))
predicciones_csv = os.path.join(current_folder, '..', 'predictions.csv')

Cargo el df con los features.

In [3]:
pd.options.mode.chained_assignment = None
%run $merge_features

KeyboardInterrupt: La limpieza ya corrió en este Kernel

KeyboardInterrupt: La limpieza ya corrió en este Kernel

KeyboardInterrupt: La limpieza ya corrió en este Kernel

In [4]:
assert(df_features.shape[0] == get_clean_df()['person'].unique().shape[0])

Cargo el set de entrenamiento.

In [5]:
%run $set_de_entrenamiento_testing_y_prediccion

labels_with_features = labels.merge(df_features, how='inner', on='person')
train = labels_with_features.drop('label', axis=1)
train_target = labels_with_features['label']

### Predictores base

En esta sección vamos a preparar los predictores base a utilizar. Estos son los mismos que se encuentran en la carpeta *Algoritmos de ML*.

In [6]:
base_predictors = []

#### Xgboost

Nota: vamos a usar XGBRegressor para tener la misma interfaz con el resto de los predictores. Los hiperparámetros tienen distinto nombre, pero producen los mismos resultados. Se puede consultar la documentación de xgboost para encontrar los nombres de los parámetros: https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.XGBRegressor. 

In [7]:
param = {
    'objective': 'reg:logistic',
    'colsamble_bylevel': 1,
    'colsamble_bytree': 1,
    'min_child_weight': 1,
    'learning_rate': 0.3,
    'max_delta_step': 5,
    'n_estimators': 25,
    'reg_lambda': 3,
    'max_depth': 9,
    'silent': True,
    'subsample': 1,
    'reg_alpha': 2,
    'gamma': 10
}

base_predictors.append(XGBRegressor(**param))

#### Random Forest

In [8]:
param = {
    'bootstrap': True,
    'max_depth': 9,
    'max_features': 37,
    'min_samples_leaf': 30,
    'min_samples_split': 7,
    'n_estimators': 784
}

base_predictors.append(RandomForestRegressor(**param))

#### AdaBoost

In [9]:
param = {
    'n_estimators': 124,
    'loss': 'linear',
    'learning_rate': 0.07,
    'base_estimator': DecisionTreeRegressor(criterion='mse', max_depth=4, max_features=None,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
}

base_predictors.append(AdaBoostRegressor(**param))

### Metafeatures

Ahora vamos a realizar una predicción por cada predictor base, y las vamos a agregar a una copia del set de entrenamiento (*train_meta*).

In [10]:
train_meta = train.copy()
for predictor in base_predictors:
    train_meta[predictor.__class__.__name__] = np.nan

Para realizar las predicciones utilizar cross validation.

In [11]:
%%time
kf = KFold(n_splits=10, shuffle=False)
for train_i, validation_i in kf.split(train):    
    for predictor in base_predictors:
        # como warm_start=False cada vez que llamo fit, el modelo se reinicia
        predictor.fit(train.iloc[train_i], train_target.iloc[train_i]) # train
        train_meta[predictor.__class__.__name__].iloc[validation_i] = predictor.predict(train.iloc[validation_i]) # predict

CPU times: user 13min 12s, sys: 56 ms, total: 13min 12s
Wall time: 13min 12s


In [12]:
train_meta.head(3)

,screen_resolution_height mean,screen_resolution_width mean,screen_resolution_height std,screen_resolution_width std,ad campaign hit,brand listing,checkout,conversion,generic listing,lead,search engine hit,searched products,...,Saturday,Sunday,Thursday,Tuesday,Wednesday,madrugada,tarde,noche,maniana,XGBRegressor,RandomForestRegressor,AdaBoostRegressor
person,,,,,,,,,,,,,,,,,,,,,,,,,
0566e9c1,568.0,320.0,0.0,0.0,6.0,3.0,1.0,1.0,15.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0,1,0,0,0.016541,0.020955,0.116183
6ec7ee77,640.0,360.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.100654,0.078044,0.293405
abe7a2fb,640.0,360.0,0.0,0.0,9.0,14.0,1.0,0.0,9.0,0.0,4.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0.010634,0.025350,0.165819


### Predictor Stacking

Ahora vamos a entrenar un nuevo modelo utilizando como features las predicciones anteriores (metafeatures). También podemos agregar algunos de los features originales.

Para esto realizamos un Random Search. TODO 

In [13]:
param = {
    'learning_rate': 0.3,
    'gamma': 1.6317896840572566,
    'max_depth': 2,
    'objective': 'reg:logistic',
    'n_estimators': 25
}

stacking = XGBRegressor(**param)
stack_train = train_meta[[predictor.__class__.__name__ for predictor in base_predictors]]

In [14]:
%%time
scores = cross_val_score(stacking, stack_train, train_target, cv=10, scoring='roc_auc')
print("Accuracy: %0.6f (+/- %0.6f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.875543 (+/- 0.022174)
CPU times: user 1.52 s, sys: 4 ms, total: 1.53 s
Wall time: 1.53 s


In [15]:
stacking.fit(stack_train, train_target)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=1.6317896840572566, learning_rate=0.3,
       max_delta_step=0, max_depth=2, min_child_weight=1, missing=None,
       n_estimators=25, n_jobs=1, nthread=None, objective='reg:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

### Bayesian Optimization

Búsque de hiperparámetros para el stacking.

In [16]:
pbounds = {
    'max_depth': (2, 20),
    'eta': (0, 0.3),
    'gamma': (0, 10),
#     'min_child_weight': (1, 5),
#     'max_delta_step': (1, 5),
#     'subsample': (0, 1),
#     'colsample_bytree': (0, 1),
#     'colsample_bylevel': (0, 1),
#     'lambda': (1, 3),
#     'alpha': (0, 2)
}

discrete = ['max_depth'] # parámetros discretos
cv_splits = 10 # cantidad de splits en el cv
num_round = 100 # cantidad máxima de boosts

In [17]:
dtrain = xgb.DMatrix(stack_train, label=train_target)
def cv_score_xgb(**param):
    param['silent'] = 1
    param['objective'] = 'reg:logistic'
    
    # transformo los valores que deben ser discretos
    for d in discrete:
        param[d] = int(param[d])
    
    # hago el cv
    scores = xgb.cv(param, dtrain, nfold=cv_splits, metrics='auc', verbose_eval=False, shuffle=False, stratified=False, num_boost_round=num_round)
    return scores['test-auc-mean'].max()

In [18]:
%%time
optimizer = BayesianOptimization(f=cv_score_xgb, pbounds=pbounds)
optimizer.probe(
    params = {'eta': 0.06027, 'gamma': 8.548, 'max_depth': 16}
)
optimizer.maximize(
    init_points=0,
    n_iter=1,
)

|   iter    |  target   |    eta    |   gamma   | max_depth |
-------------------------------------------------------------
|  1        |  0.8753   |  0.06027  |  8.548    |  16.0     |
|  2        |  0.8755   |  0.2056   |  6.063    |  4.159    |
CPU times: user 38.6 s, sys: 172 ms, total: 38.8 s
Wall time: 38.6 s


In [19]:
optimizer.max

{'params': {'eta': 0.20560106670266734,
  'gamma': 6.063466743530106,
  'max_depth': 4.158627203617048},
 'target': 0.8754747}

Escribo el mejor resultado en un archivo

In [20]:
hyperparameter_data = {
    'algorithm': 'stacking',
    'hyperparameters': optimizer.max['params'],
    'cv_splits': cv_splits,
    'auc': optimizer.max['target'],
    'features': train.columns
}

In [21]:
%run -i write_hyperparameters.py

### Predicción del set testing con los modelos base

Utilizamos los modelos entrenados con el 100% del set de entrenamiento para predecir el set de testing.

In [22]:
testing = labels_to_predict.merge(df_features, how='inner', on='person')
assert(testing.shape[0] == labels_to_predict.shape[0])

In [23]:
testing_with_base_predictions = testing.copy()
for predictor in base_predictors:
    testing_with_base_predictions[predictor.__class__.__name__] = np.nan

In [24]:
%%time
for predictor in base_predictors:
    # como warm_start=False cada vez que llamo fit, el modelo se reinicia
    predictor.fit(train, train_target) # train
    testing_with_base_predictions[predictor.__class__.__name__] = predictor.predict(testing) # predict

CPU times: user 1min 29s, sys: 24 ms, total: 1min 29s
Wall time: 1min 29s


### Predicción del set testing con el stacking

Utilizamos las predicciones como features para que el Stacking las combine en una prediccón final.

In [25]:
testing_with_base_predictions_for_stacking = testing_with_base_predictions[[predictor.__class__.__name__ for predictor in base_predictors]]
testing_with_base_predictions_for_stacking.head(3)

,XGBRegressor,RandomForestRegressor,AdaBoostRegressor
person,,,
4886f805,0.004958,0.002692,0.037771
0297fc1e,0.027957,0.067237,0.158337
2d681dd8,0.009751,0.003750,0.037771


In [26]:
predictions = stacking.predict(testing_with_base_predictions_for_stacking)

In [27]:
testing_target = pd.DataFrame(data=stacking.predict(testing_with_base_predictions_for_stacking))
testing_target.index = testing_with_base_predictions_for_stacking.index
testing_target.index.name = 'person'
testing_target.columns = ['label']

In [28]:
# testing_target.to_csv(predicciones_csv)